In [0]:
#!usr/bin/env python3
# -*- coding: utf-8 -*-

In [2]:
# インスタンス起動からの時間を表示(0.5days: 12hでセッションの有無にかかわらずシャットダウンされる)
# タブがアイドル状態で90mが経過してもシャットダウンされる
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'

0.000870833days (75.24sec)


In [3]:
# 残りのディスク容量を確認
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         359G   26G  315G   8% /
tmpfs           6.4G     0  6.4G   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
tmpfs           6.4G   12K  6.4G   1% /var/colab
/dev/sda1       365G   31G  335G   9% /opt/bin
shm             6.0G     0  6.0G   0% /dev/shm
tmpfs           6.4G     0  6.4G   0% /sys/firmware


In [4]:
# メインメモリの利用状況を確認(70~80%を超えるとダイアログで警告される)
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        409M         10G        908K        1.8G         12G
Swap:            0B          0B          0B


In [5]:
# GPUメモリの利用状況確認
!nvidia-smi

Thu Jun 13 20:58:39 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    18W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls 'drive/My Drive/mnt'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
const_for_colab.py		   spectrograms_old  util_for_colab.py
processing_DSD100_for_colab.ipynb  src
spectrograms			   tf-layers-model


In [3]:
# ファイルのアップロード
from google.colab import files
uploaded = files.upload()

Saving const_for_colab.py to const_for_colab.py


In [0]:
!rm const_for_colab.py

In [4]:
!ls

const_for_colab.py  drive  __pycache__	sample_data  util_for_colab.py


In [0]:
# ファイルのダウンロード
files.download('./tf-layers-model/model.ckpt-1.meta')

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import concatenate
import numpy as np
import const_for_colab as C
import util_for_colab as util
import os

# GPUがアサインされているか確認
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
# tf.nnのfilter = [kernel_height, kernel_width, input_channel, output_channel]
# 畳み込み層(エンコーダ部分)
def conv2d(x, output_channel, kernel=5, stride=2, pad='same', batch_norm=True, leaky_relu=True):
    net = tf.layers.conv2d(x,
                           filters=output_channel,
                           kernel_size=[kernel, kernel],
                           strides=[stride, stride],
                           padding=pad)
    if leaky_relu:
        net = tf.nn.leaky_relu(net, 0.2)
    if batch_norm:
        net = tf.layers.batch_normalization(net)
    return net

In [0]:
# tf.nnのfilter = [kernel_height, kernel_width, output_channel, input_channel]
# output_shape = [バッチ数, 得たいheight, 得たいwidth, 得たいchannel]
# 逆畳み込み層(デコーダ部分)
def de_conv2d(x, output_channel, kernel=5, stride=2, pad='same', batch_norm=True, relu=True):
    net = tf.layers.conv2d_transpose(x,
                                     filters=output_channel,
                                     kernel_size=[kernel, kernel],
                                     strides=[stride, stride],
                                     padding=pad)
    if relu:
        net = tf.nn.relu(net)
    if batch_norm:
        net = tf.layers.batch_normalization(net)
    return net

In [0]:
def U_Net(features, labels, mode):
    tf_x = tf.reshape(features['mag'], [-1, 512, 128, 1])
    conv1 = conv2d(tf_x, output_channel=16)
    conv2 = conv2d(conv1, output_channel=32)
    conv3 = conv2d(conv2, output_channel=64)
    conv4 = conv2d(conv3, output_channel=128)
    conv5 = conv2d(conv4, output_channel=256)
    conv6 = conv2d(conv5, output_channel=512)
    de_conv1 = de_conv2d(conv6, output_channel=256)
    dropout1 = tf.layers.dropout(de_conv1, rate=0.5, training=(mode == tf.estimator.ModeKeys.TRAIN))
    de_conv2 = de_conv2d(concatenate([dropout1, conv5], 3), output_channel=128)
    dropout2 = tf.layers.dropout(de_conv2, rate=0.5, training=(mode == tf.estimator.ModeKeys.TRAIN))
    de_conv3 = de_conv2d(concatenate([dropout2, conv4], 3), output_channel=64)
    dropout3 = tf.layers.dropout(de_conv3, rate=0.5, training=(mode == tf.estimator.ModeKeys.TRAIN))
    de_conv4 = de_conv2d(concatenate([dropout3, conv3], 3), output_channel=32)
    de_conv5 = de_conv2d(concatenate([de_conv4, conv2], 3), output_channel=16)
    de_conv6 = de_conv2d(concatenate([de_conv5, conv1], 3), output_channel=1, relu=False)
    activation_final = tf.math.sigmoid(de_conv6)

    prediction = {'outputs': activation_final}

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # 損失関数
    loss = tf.losses.absolute_difference(labels, activation_final)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(1e-4)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    return tf.estimator.EstimatorSpec(mode=mode, loss=loss)

def train():
    X_list, y_list = util.load_dataset('drive/My Drive/mnt/spectrograms', target="vocal")
    X_magnitude, X_phase = util.magnitude_phase(X_list)
    y_magnitude, _ = util.magnitude_phase(y_list)
    
    deep_u_net = tf.estimator.Estimator(model_fn=U_Net, model_dir='./model')
    
    for e in range(40):
        X, y = util.sampling(X_magnitude, y_magnitude)
        train_input_fn = tf.estimator.inputs.numpy_input_fn(x={'mag':X}, y=y,
                                                            batch_size=C.BATCH_SIZE, num_epochs=1, shuffle=False)
        deep_u_net.train(input_fn=train_input_fn)
        print('training epoch: ' + str(e))

In [0]:
train()

In [0]:
file_name = "UNION_cut.wav"
mag, phase = util.load_audio(file_name)
start = 60
end = start + C.PATCH_SIZE

mag = mag[:, start:end]
phase = phase[:, start:end]

X = mag[1:].reshape(1,512,128,1)
    
predict_input_fn = tf.estimator.inputs.numpy_input_fn(x = {'mag':X},y = None,num_epochs = 1,shuffle = False)

deep_u_net = tf.estimator.Estimator(model_fn=U_Net, model_dir="./model")
predictions = list(deep_u_net.predict(input_fn=predict_input_fn))
mask = predictions[0]['outputs']

target_pred_mag = np.vstack((np.zeros((128)), mask.reshape(512, 128)))

util.save_audio(file_name[:-4]+"_vocal.wav", target_pred_mag, phase)
util.save_audio(file_name[:-4]+"_inst.wav", mag - target_pred_mag, phase)
util.save_audio(file_name[:-4]+"_orig.wav", mag, phase)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.conv2d_transpose instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loading model from drive/mnt/tf-layers-model/
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from drive/mnt/tf-layers-model/model.ckpt-1


TypeError: ignored